In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import norm, skew # 수치해석 기능(확률분포) 제공
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,roc_curve,roc_auc_score,log_loss

train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/test.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/천체 유형 분류/sample_submission.csv')

train_id = train['id']
test_id = test['id']

# 필요없는 Id 컬럼, Drop하자.
#train.drop('id',axis=1,inplace=True)
#test.drop('id',axis=1,inplace=True)

# 'Id'를 제거한 DataFrame의 Shape 확인
print("The train data size after dropping ID feature is : {} ".format(train.shape))
print("The test data size after dropping Id feature is : {} ".format(test.shape))

unique_labels = train['type'].unique()
label_dict = {val: i for i, val in enumerate(unique_labels)}
i2lb = {v:k for k, v in label_dict.items()}

The train data size after dropping ID feature is : (199991, 23) 
The test data size after dropping Id feature is : (10009, 22) 


In [0]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [4]:
test_id=test['id']
test = test.drop(columns=['fiberID', 'id'])

train_x = train.drop(columns=['id','type_num', 'fiberID','type'], axis=1)
train_y = train['type_num']
test_x = test
train_x

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,20.630780,20.437920,20.050756,20.020974,19.959646,20.948519,20.726709,20.311562,20.323418,20.211336,20.836400,20.536338,20.126561,20.102815,19.796722,20.783295,20.326879,20.014046,20.061940,19.862513
199987,25.283695,22.807243,21.069792,20.187531,19.595598,26.677671,22.818723,20.786120,19.961857,19.457071,28.895442,22.250489,20.091693,18.985791,18.803354,26.921903,21.952440,19.776321,18.990039,18.528306
199988,23.609826,21.902709,20.330747,18.457535,17.326771,24.320816,22.287946,20.711737,18.810560,17.721928,25.982378,22.132993,20.329624,18.507337,17.418521,24.503460,21.897599,20.420524,18.443126,17.331724
199989,21.172003,20.561208,20.031754,19.793644,19.540471,21.704126,20.469748,19.872089,19.603742,19.372909,18.880781,18.065559,17.501307,17.180639,17.175205,19.109086,18.004976,17.440331,17.193743,16.969854


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, \
                                                  test_size=0.2, random_state=42, stratify = train_y)

#RF

In [6]:
%%time
rnd_clf = RandomForestClassifier(max_depth=20, n_estimators=1000,max_features=10, oob_score=False, n_jobs=-1)
rnd_clf.fit(X_train,y_train)
rnd_clf_probs = rnd_clf.predict_proba(X_valid)
score = log_loss(y_valid, rnd_clf_probs)
print(score)

0.3913917938751966
CPU times: user 1h 15min 31s, sys: 7.41 s, total: 1h 15min 38s
Wall time: 38min 13s


# XGB

In [8]:
%%time
from xgboost import XGBClassifier

XGB_clf = XGBClassifier(n_esetimators=1500,
                        n_jobs=-1,
                        max_depth=10,
                        scale_pos=10,
                        random_state=0)
XGB_clf.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=20)

[0]	validation_0-merror:0.133269	validation_1-merror:0.156729
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 20 rounds.
[1]	validation_0-merror:0.125988	validation_1-merror:0.151404
[2]	validation_0-merror:0.122675	validation_1-merror:0.150079
[3]	validation_0-merror:0.119675	validation_1-merror:0.148879
[4]	validation_0-merror:0.117656	validation_1-merror:0.148304
[5]	validation_0-merror:0.115156	validation_1-merror:0.147604
[6]	validation_0-merror:0.113781	validation_1-merror:0.147354
[7]	validation_0-merror:0.112124	validation_1-merror:0.146679
[8]	validation_0-merror:0.110387	validation_1-merror:0.146404
[9]	validation_0-merror:0.109162	validation_1-merror:0.146079
[10]	validation_0-merror:0.108112	validation_1-merror:0.144904
[11]	validation_0-merror:0.106674	validation_1-merror:0.144529
[12]	validation_0-merror:0.105587	validation_1-merror:0.144304
[13]	validation_0-merror:0.1

#LGB

In [61]:
from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(boosting_type='gbdt', num_leaves=250, max_depth=-1, learning_rate=0.005, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)

evals =[(X_train,y_train),(X_valid,y_valid)]
lgbm_wrapper.fit(X_train,y_train,early_stopping_rounds=20,
                 eval_metric='logloss',eval_set=evals,verbose=True)

#scores_lda = cross_val_score(lgbm_wrapper,data_lda,y_data,scoring='neg_log_loss',cv=5)
#print('평균 f1-score:{0:.4f}'.format(np.mean(scores_lda)))

#preds = lgbm_wrapper.predict_proba(test_lda)

[1]	training's multi_logloss: 2.27262	training's multi_logloss: 2.27262	valid_1's multi_logloss: 2.27573	valid_1's multi_logloss: 2.27573
Training until validation scores don't improve for 20 rounds.
[2]	training's multi_logloss: 2.24591	training's multi_logloss: 2.24591	valid_1's multi_logloss: 2.25152	valid_1's multi_logloss: 2.25152
[3]	training's multi_logloss: 2.22099	training's multi_logloss: 2.22099	valid_1's multi_logloss: 2.22875	valid_1's multi_logloss: 2.22875
[4]	training's multi_logloss: 2.19745	training's multi_logloss: 2.19745	valid_1's multi_logloss: 2.20719	valid_1's multi_logloss: 2.20719
[5]	training's multi_logloss: 2.17502	training's multi_logloss: 2.17502	valid_1's multi_logloss: 2.18655	valid_1's multi_logloss: 2.18655
[6]	training's multi_logloss: 2.15354	training's multi_logloss: 2.15354	valid_1's multi_logloss: 2.16681	valid_1's multi_logloss: 2.16681
[7]	training's multi_logloss: 2.13292	training's multi_logloss: 2.13292	valid_1's multi_logloss: 2.1478	valid_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.005, max_depth=-1,
               min_child_samples=17, min_child_weight=0.005, min_data_in_leaf=5,
               min_split_gain=0.34, n_estimators=10000, n_jobs=-1,
               num_leaves=250, objective='multiclass', random_state=123,
               reg_alpha=0.01, reg_lambda=0.01, silent=True, subsample=1.0,
               subsample_for_bin=250000, subsample_freq=0)

# Stacking

In [0]:
rnd_final = RandomForestClassifier(max_depth=5, n_estimators=1500, oob_score=False, n_jobs=-1)
#from sklearn.linear_model import LogisticRegression
#lr_final = LogisticRegression(C=10)

In [0]:
rf_pred = rnd_clf.predict(X_valid)
xgb_pred = XGB_clf.predict(X_valid)
lgb_pred = lgbm_wrapper.predict(X_valid)

In [26]:
#print('rf',log_loss(y_valid,rf_pred))
#print('xgb',log_loss(y_valid,xgb_pred))

rf 0.3913917938751966
xgb 0.39698752170559687


In [67]:
pred = np.array([rf_pred,xgb_pred,lgb_pred])
print(pred.shape)

pred = np.transpose(pred)
print(pred.shape)

(3, 39999)
(39999, 3)


In [68]:
rnd_final.fit(pred,y_valid)
final = rnd_final.predict_proba(pred)

print('rnd_final',log_loss(y_valid,final))

rnd_final 0.5763971076300051


# Stratify_Stacking

In [86]:
y_valid.describe()

count    39999.000000
mean         9.877372
std          4.624128
min          1.000000
25%          7.000000
50%          9.000000
75%         12.000000
max         19.000000
Name: type_num, dtype: float64

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold

def get_stacking_data(model,X_train_n,y_train_n,X_test_n,n_folds):
  # 지정된 n_folds값으로 kFold 생성.
  kf = KFold(n_splits=n_folds,shuffle=False)
  #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
  train_fold_pred = np.zeros((X_train_n.shape[0],1))
  test_pred = np.zeros((X_test_n.shape[0],n_folds))
  print(model.__class__.__name__,'model 시작')

  for folder_counter, (train_index,valid_index) in enumerate(kf.split(X_train_n)):
    # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
    print('\t 폴드 세트:',folder_counter,'시작')
    X_tr = X_train_n.iloc[train_index]
    y_tr = y_train_n.iloc[train_index]
    X_te = X_train_n.iloc[valid_index]

    #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
    model.fit(X_tr,y_tr)
    #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
    train_fold_pred[valid_index,:] = model.predict(X_te).reshape(-1,1)
    #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장.
    test_pred[:,folder_counter] = model.predict(X_test_n)

  #폴드 세트내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터 생성
  test_pred_mean = np.mean(test_pred,axis=1).reshape(-1,1)

  #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
  return train_fold_pred,test_pred_mean

In [116]:
XGB_train,XGB_test = get_stacking_data(XGB_clf,X_train,y_train,X_valid,3)
rnd_train,rnd_test = get_stacking_data(rnd_clf,X_train,y_train,X_valid,3)

XGBClassifier model 시작
	 폴드 세트: 0 시작
	 폴드 세트: 1 시작
	 폴드 세트: 2 시작
RandomForestClassifier model 시작
	 폴드 세트: 0 시작
	 폴드 세트: 1 시작
	 폴드 세트: 2 시작


In [0]:
stack_final_X_train = np.concatenate((XGB_train,rnd_train),axis=1)
stack_final_X_test = np.concatenate((XGB_test,rnd_test),axis=1)

In [120]:
stack_final_X_train

array([[19., 19.],
       [ 7.,  7.],
       [ 7.,  7.],
       ...,
       [ 8., 19.],
       [16., 16.],
       [ 9.,  9.]])

In [123]:
pred = lgbm_wrapper.fit(stack_final_X_train,y_train)
stack_final = lgbm_wrapper.predict_proba(stack_final_X_test)

print(log_loss(y_valid,stack_final))

0.6558941269720046
